每日主題知識點
● 多元線性回歸

甚麼是多元線性回歸
● 多元線性回歸
還記得前兩章我們用單變數來做線性回歸?相信大家都覺得太無趣了，這麼簡單用手算就好了。所以今天我們要往前邁進一步。

● 今日目標
如果我們一次拿多個變數來處理，包括資料整理、設定損失函數與梯度下降等等，情況會變得非常複雜!今天我們就專注於處理多個變數的線性回歸吧!

● 我們本日一樣採用糖尿病資料集，但除了年齡，再多加入一筆 BMI 數值(在數據中名字為 bmi)。

● 比起兩天前的程式碼，獲得兩筆變數的數據(糖尿病數據集)，只不過多了一個步驟。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
from sklearn.datasets import load_boston

diabetes = load_diabetes()
data, yt = diabetes.data, diabetes.target  # data 每一筆資料為一間房子 yt為對應到data的疾病概況
feature_names = diabetes.feature_names
print('輸入資料', data.shape, yt.shape)
print('資料的特徵名', feature_names)

# 單看 age(年齡) 這筆特徵
x_data = data[:, feature_names.index('age')].reshape(-1, 1)
print('整理後', x_data.shape)

# 在 x 向量第 1 個位置中加入虛擬變數
x = np.insert(x_data, 0, 1.0, axis=1)
print('加入虛擬變數後', x.shape)

# 整理完後的資料長相
print(x.shape)

# 看看前五位病人之年齡(加上虛擬變數)如何對應疾病概況
print(x[:5,:])
print(yt[:5])
輸出如下:
輸入資料 (442, 10) (442,)
特徵名 ['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']
整理後 (442, 1)
加入虛擬變數後 (442, 2)
(442, 2)
[[ 1. 0.03807591]
[ 1. -0.00188202]
[ 1. 0.08529891]
[ 1. -0.08906294]
[ 1. 0.00538306]]

● 我們這次再追加一個變數(BMI):

# 用 hstack() 追加列（BMI）
x_add = data[:,feature_names.index('bmi')].reshape(-1, 1)
x2 = np.hstack((x, x_add))
print(x2.shape)

# 輸出輸入資料（含虛擬資料）
print(x2[:5,:])

# 輸出實際值
print(yt[:5])
輸出如下:
(442, 3)
[[ 1. 0.03807591 0.06169621]
[ 1. -0.00188202 -0.05147406]
[ 1. 0.08529891 0.04445121]
[ 1. -0.08906294 -0.01159501]
[ 1. 0.00538306 -0.03638469]]

實作多元線性迴歸(I)
● 首先，了解必須清楚我們想採取的的模型長相(如圖1)，以及預測函數、資料集長相(如圖2)與損失函數還有優化演算法。

● 模型長相
首先要確定輸入及輸出。
多元線性迴歸.png
圖1

● 預測函數
 
 
 
 
 
 
 

● 資料長相
image
圖2

● 損失函數
 
 
 
 
 
 
 
 

這裡的上標 m 代表第 m 筆資料(如圖1中左邊的index)。
這裡已經幫大家算好這個損失函數的偏微分，如下

 
 
 
 
 
 
 
 
 

這裡 
  是第 m 筆數據之誤差(預測值
真實值)

 
 
 
 
 
 
 
 
 
 

● 優化演算法

 
 
 

 
 
 

 
 
 
 
 
 

這裡
為學習率(learning rate，可參照機器學習馬拉松)。

實作多元線性回歸(II)
● 我們用程式驗證看看上一頁的數學分析吧!

● 預測函數

# 以預測函數 (1, x) 之值計算預測值 yp
def pred(x, w):
    return(x @ w)
● 初始化

# 初始化處理

# 資料樣本總數
M = x2.shape[0]

# 輸入資料之維數（含虛擬變數）
D = x2.shape[1]

# 迭代運算次數
iters = 50000
# iters = 2000

# 學習率
alpha = 0.01
# alpha = 0.001

# 權重向量的初始值（預設全部為 1）
w = np.ones(D)

# 記錄評估結果用（僅記錄損失函數值）
history = np.zeros((0,2))
● 迭代運算

# 迭代運算
for k in range(iters):
    
    # 計算預測值（7.8.1）
    yp = pred(x2, w)
    
    # 計算誤差（7.8.2）
    yd = yp - yt
    
    # 梯度下降法的實作（7.8.4）
    w = w - alpha * (x2.T @ yd) / M
    
    # 繪製學習曲線所需資料之計算與儲存
    if ( k % 100 == 0):
        # 計算損失函數值（7.6.1）
        loss = np.mean(yd ** 2) / 2
        # 記錄計算結果
        history = np.vstack((history, np.array([k, loss])))
        # 顯示畫面
#        print( "iter = %d  loss = %f" % (k, loss))
● 查看輸出

# 損失函數的初始值、最終值
print('損失函數初始值: %f' % history[0,1])
print('損失函數最終值: %f' % history[-1,1])
輸出結果:
損失函數初始值: 14382.773907
損失函數最終值: 2023.238005

● 調整超參數後，運行迭代及查看結果 (把學習率降為
，把迭代次數降為
)，記得重新迭代，要重設權重值(已被更新過)喔!

# 降低迭代次數
iters = 2000

# 降低學習率
alpha = 0.001

# 權重向量的初始值（預設全部為 1）
w = np.ones(D)

# 記錄評估結果用（僅記錄損失函數值）
history = np.zeros((0,2))

# 迭代運算
for k in range(iters):
    
    # 計算預測值（7.8.1）
    yp = pred(x2, w)
    
    # 計算誤差（7.8.2）
    yd = yp - yt
    
    # 梯度下降法的實作（7.8.4）
    w = w - alpha * (x2.T @ yd) / M
    
    # 繪製學習曲線所需資料之計算與儲存
    if ( k % 100 == 0):
        # 計算損失函數值（7.6.1）
        loss = np.mean(yd ** 2) / 2
        # 記錄計算結果
        history = np.vstack((history, np.array([k, loss])))
        # 顯示畫面
#        print( "iter = %d  loss = %f" % (k, loss))

# 損失函數的初始值、最終值
print('損失函數初始值: %f' % history[0,1])
print('損失函數最終值: %f' % history[-1,1])
輸出結果:
損失函數初始值: 14382.773907
損失函數最終值: 3207.520633

● 可視化我們的訓練過程吧

# 繪製學習曲線（第一組數除外）
plt.plot(history[:,0], history[:,1])
plt.xlabel('iterations')
plt.ylabel('loss')
plt.show()
知識點總整理
● 多元線性回歸

多個變數組成的線性回歸問題，在計算上較複雜，建議同學熟悉前一天的數學概念，對深度學習也非常有用。

延伸閱讀
● 在此分享有關學習率的文章，建議本課程與機器學習馬拉松同時進行，更能讓自己在數學簡單分析能力與程式能力做結合。

https://www.plob.org/article/21083.html

